In [1]:
import torch
from utils.networks import return_resnet18
from utils.dataloader import return_dataloaders
from glob import glob

In [2]:
dataset = 'bffhq'
batch_size = 4
num_BDC = 5
PBA = []
for BCD_idx in range(num_BDC):
    BCD = return_resnet18(num_classes=2)
    # BCD.load_state_dict(torch.load(f'./models/{dataset}/BCD{BCD_idx}.pth'))
    BCD.eval()
    PBA.append(BCD)

In [ ]:
import torch.nn as nn
import pandas as pd
bias_prob_threshold = 0.5
softmax = nn.Softmax(dim=1)
dataloaders = return_dataloaders(dataset=dataset,
                                root='../../dataset/',
                                batch_size=batch_size,
                                shuffle=True)

for batch_idx, (X, y, path, *_) in enumerate(dataloaders['train']):
    gt_probs_list = []
    for BCD_idx in range(num_BDC):
        logits = PBA[BCD_idx](X)
        probs = softmax(logits)
        print(probs)
        result = probs.gather(1, y.unsqueeze(1)).squeeze(1)
        gt_probs_list.append(result)

    gt_probs = torch.stack(gt_probs_list, dim=0)

    # p_y가 bias_prob_threshold를 초과하도록 예측한 BCD가 절반 이상이면 biased(True)
    # 그렇지 않으면 conflict(False)
    bias_flag = (gt_probs>bias_prob_threshold).sum(dim=0)>(num_BDC/2)
    int_bias_flag = bias_flag.long()
    list_bias_flag = int_bias_flag.tolist()

    batch_bias_table = {
        'file_path': path,
        'bias': list_bias_flag
    }
    df = pd.DataFrame(batch_bias_table)
    df.to_csv(f'./bias_tables/{dataset}/bias_table_{batch_idx}.csv', index=False)

    if batch_idx == 3:
        break

In [4]:
bias_tables = glob(f'./bias_tables/{dataset}/bias_table_*.csv')
dfs = [pd.read_csv(f) for f in bias_tables]
df = pd.concat(dfs, ignore_index=True)
df.to_csv(f'./bias_tables/{dataset}/bias_table.csv', index=False)